# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/sudhanagrath/Documents/dataengineering/ApacheCassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1595


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
#Connecting to a Cassandra instance running on my local machine

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

#Create a session to connect, load and query the data on the Instance
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 
query="CREATE KEYSPACE udacity WITH REPLICATION" + "= {'class':'SimpleStrategy', 'replication_factor':1};";
session.execute(query)

#### Set Keyspace

In [6]:
# Set KEYSPACE to the keyspace specified above
session.execute("USE udacity");

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
##Creating a table based on the query1 to  retrieve the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1="CREATE TABLE IF NOT EXISTS song_history"
query1=query1 + "(sessionId int, itemInSession int, artist varchar, \
       song varchar, length float,PRIMARY KEY(sessionId, itemInSession))"

session.execute(query1)    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Inserting data into the table song_history
        query = "INSERT INTO song_history(sessionId, itemInSession, \
        artist, song, length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        #For the current line in CSV file, select delimited fields in the\
        #specified order of columns in the INSERT statement and execute \
        #the statement
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
##Checking the data as per query1 to ensure if the data has been loaded 
query="SELECT artist, song, length,sessionid, iteminsession FROM song_history \
        WHERE sessionId=338 and itemInSession=4"
records=session.execute(query)
for record in records:
    print(record.artist, record.song, record.length, record.sessionid, record.iteminsession)
    


Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [12]:
##Creating a table based on the query2 to retrieve the name of artist,\
##song (sorted by itemInSession) and user (first and last name)\
##for userid = 10, sessionid = 182
query2="CREATE TABLE IF NOT EXISTS user_artist_history"
query2=query2 +  "(userId int, sessionId int,itemInSession int,\
        artist varchar, song varchar, firstName varchar,lastName varchar,\
         PRIMARY KEY((userId, sessionId),itemInsession))"

session.execute(query2) 
                    

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## INSERTING data into user_artist_history
        query = "INSERT INTO user_artist_history(userId, sessionId, itemInSession,\
        artist, song, firstName,lastName)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s)"
        #For the current line in CSV file, select delimited fields in the\
        #specified order of columns in the above INSERT statement and execute \
        #the statement
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),\
                                line[0],line[9],line[1],line[4]))

In [18]:
## Checking the data as per query2 to ensure if the data has been loaded 
query="SELECT artist, song, firstName, lastName, userid, sessionid,\
       iteminsession FROM user_artist_history WHERE userId=10 \
       and sessionId=182"
records=session.execute(query)
for record in records:
    print(record.artist, record.song, record.firstname, record.lastname, \
          record.userid,record.sessionid, record.iteminsession)


Down To The Bone Keep On Keepin' On Sylvie Cruz 10 182 0
Three Drives Greece 2000 Sylvie Cruz 10 182 1
Sebastien Tellier Kilometer Sylvie Cruz 10 182 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 10 182 3


In [19]:
## Creating a table based on the query3 to retrieve every user name \
##(first and last) in my music app history who listened to the song \
##'All Hands Against His Own'
query3="CREATE TABLE IF NOT EXISTS user_info"
query3=query3 + "(song varchar,userId int, firstName varchar, \
      lastName varchar,PRIMARY KEY(song, userId))"
session.execute(query3)                   

In [20]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Inserting data into user_info table
        query = "INSERT INTO user_info(song, userId,firstName,lastName)"
        query = query + "VALUES(%s,%s,%s,%s)"
        #For the current line in CSV file, select delimited fields in the\
        #specified order of columns in the above INSERT statement and execute \
        #the statement
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [22]:
##Checking the data as per query3 to ensure if the data has been loaded.
query="SELECT userid, firstName, lastName, song FROM user_info WHERE song='All Hands Against His Own' "
records=session.execute(query)
for record in records:
    print(record.userid, record.firstname, record.lastname,record.song)
    



95 Sara Johnson All Hands Against His Own


### Drop the tables before closing out the sessions

In [23]:
## Dropping the 3 tables
query4="DROP TABLE IF EXISTS song_history"
query5="DROP TABLE IF EXISTS user_artist_history"
query6="DROP TABLE IF EXISTS user_info"
session.execute(query4)
session.execute(query5)
session.execute(query6)

In [37]:
#session.execute("DROP KEYSPACE udacity")

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()